In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int

In [2]:


class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


In [20]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")


class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    
#%% testing qsgd quantization

print("QSGD")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = QSGD(2,True)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")



QSGD
Before Quantization:  [2.926      1.4817569  1.5193974  9.264953   0.97082645 8.989641
 0.46480295 3.5107994  4.772718   3.911427   5.6062117  5.7005963
 9.021768   8.99285    9.949473   2.5060265 ]
Encoded b'\'\x00\x00\x00\x16\xc7\xbdA2Ye"\x02\x00\x00\x00'
After Quantization:  [11.861105  0.        0.       11.861105  0.        0.        0.
  0.        0.       11.861105  0.        0.       11.861105 11.861105
 11.861105  0.      ]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [19]:
#%%SETTING UP Julia and gzip Class
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/gzip.jl")

import numpy as np

class GZip(Compressor):
    def __init__(self, s: int):
        self.s = s

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_gzip(array, self.s, seed)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_gzip(array, self.s)
        return result


#%% testing gzip quantization
print("GZip")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = GZip(1)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


GZip
Before Quantization:  [3.5150225  9.831025   2.0403607  7.4923944  4.3590503  1.1596535
 3.2800493  4.2256823  6.228186   3.0646484  0.35763338 0.8877172
 4.2105975  1.6903234  4.103967   9.747695  ]
Encoded b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\ncdd\x00\x02F0\x02\x01\xbbo\xf3\x1d\x01\x06yn\xb7\x14\x00\x00\x00'
After Quantization:  [19.995235 19.995235  0.        0.        0.        0.       19.995235
  0.        0.       19.995235  0.        0.        0.        0.
  0.        0.      ]
37.5% smaller 
61.30952380952381% smaller 


In [18]:
#%%FP8

# print("Starting up Julia.")
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/fp8.jl")
# print("Finished starting up Julia.")

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)


class FP8(Compressor):
    def __init__(self):
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
#%% testing fp32 quantization
print("FP8")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = FP8()
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


FP8
Before Quantization:  [7.8672147  8.451869   8.442888   7.9562845  9.411241   8.508729
 7.513384   2.8971407  8.685831   6.922977   9.248251   7.1175513
 7.9733496  1.364026   5.5880284  0.47929356]
Encoded b'HHHHIHHAHGHGH=F7'
After Quantization:  [ 8.      8.      8.      8.     10.      8.      8.      2.5     8.
  7.      8.      7.      8.      1.25    6.      0.4375]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [21]:

import cv2


class customCompression():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    



#%%

# data = np.random.rand(1,16)

data = list(np.random.rand(1,16).flatten()*10)
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint8
cmp = customCompression(data_type)
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))


[9.329273   1.8753529  8.997384   9.441993   5.0720806  7.441941
 1.7281626  7.655823   8.453056   1.1371003  0.9238816  9.737485
 0.62521195 5.950775   6.581495   1.2450198 ]
Encoded [243  34 234 246 124 190  30 196 219  14   8 255   0 149 166  17]
After Quantization:  [9.30867209 1.84018168 8.98706245 9.4158753  5.05627803 7.41474868
 1.69724407 7.62915511 8.45104639 1.12549361 0.91108718 9.73748493
 0.62521195 5.94963813 6.55712299 1.23269682]
<class 'numpy.uint8'> 48.275862068965516% smaller 
<class 'numpy.uint8'> 28.571428571428573% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  0.00038474639469928833


In [7]:
print(sys.getsizeof(data))
print(sys.getsizeof(dec))
print(sys.getsizeof(enc))

168
232
136


# Custom Frequency Domain Quantization

In [15]:
from scipy.fftpack import dct, idct

class customFreq():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.15201865 0.94785595 0.7547502  0.58919585 0.31936973 0.771123
 0.70062065 0.06526448 0.18222547 0.54540354 0.05359937 0.17129524
 0.5470244  0.87269574 0.41106632 0.8752189 ]
Encoded [0.05902 0.9307  1.057   0.8057  0.4634  0.697   0.782   0.08325 0.0958
 0.565   0.08527 0.272   0.646   0.6626  0.07294 0.4048 ]
After Quantization:  [0.15192075 0.9480067  0.7549934  0.5889955  0.31951672 0.770956
 0.7007883  0.06520161 0.18229091 0.5452986  0.05363014 0.17113434
 0.5470309  0.87272555 0.41109377 0.87529826]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  1.6453559e-08


# Custom Freq 2 (CustomFreq + Custom2)

In [22]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.70678    0.25559768 0.20349394 0.37540936 0.73544997 0.8823482
 0.44973618 0.20250602 0.8378306  0.46143174 0.8559715  0.6128923
 0.54620606 0.3034762  0.5111137  0.37186486]
Encoded [3329568385 2677497095 1910410746 2202664210 3885809169 4294967295
 3006877142  882269678 3812976425 2088005193 2769512143 1518510068
  596997677          0  728294004   28855464]
After Quantization:  [0.70677989 0.25559764 0.20349386 0.37540935 0.73544987 0.88234821
 0.44973621 0.20250598 0.83783045 0.46143175 0.85597145 0.61289234
 0.54620599 0.3034762  0.51111372 0.37186483]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  4.356403983786257e-15


In [23]:
#%% PAQ with Compression


class MNIST_PAQ_COMP:
	def __init__(self, cmp, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename
		self.compression_ratio = 0
		self.cmp=cmp

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	def get_weights_custom(self, dtype=np.float32):
        
		precision = self.precision 
		weights = []
		start_size = 0
		end_size = 0
        
        
		for layer in self.model:			
			try:
				layer_weights = layer.weight.detach().numpy().astype(dtype)
				start_size = start_size + sys.getsizeof(layer_weights)
				# layer_weights_enc = self.cmp.encode(layer_weights)
				layer_weights_enc = self.cmp.encode(layer_weights.flatten())
				decoded_weights = self.cmp.decode(layer_weights_enc)
				decoded_weights = decoded_weights.astype(dtype)
				decoded_weights = decoded_weights.reshape(layer_weights.shape)
				end_size = end_size + sys.getsizeof(decoded_weights)
		
				layer_bias = layer.bias.detach().numpy().astype(dtype)
				# layer_bias_enc = self.cmp.encode(layer_bias)	
				layer_bias_enc = self.cmp.encode(layer_bias.flatten())			
				decoded_bias = self.cmp.decode(layer_bias_enc)
				decoded_bias = decoded_bias.astype(dtype)
				decoded_bias = decoded_bias.reshape(layer_bias.shape)
				weights.append([decoded_weights,decoded_bias])
				
			except:
				continue
		self.compression_ratio = ((start_size-end_size)/start_size)*100
		return np.array(weights),start_size,end_size



	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
        
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights

	def client_generator(self, train_x, train_y):
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		# weights,start_size,end_size = self.get_weights()
		weights,start_size,end_size = self.get_weights_custom()
		
		return weights, running_loss, start_size,end_size
	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for dataset_x, dataset_y in clients:
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss, start_size,end_size = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			agg_weight = self.set_weights(aggregate_weights)
			test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(test_acc.item(), 5))
			print("Compression Ratio ", self.compression_ratio)
			# print()
			print(f'start_size: {start_size/1024},end_size: {end_size}')
			clients.close()
		return agg_weight

#%%

In [24]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
epoch_time = time.time()
tic = time.time()
filename = f"saved_models_{epoch_time}"
train_x, train_y, test_x, test_y = Dataset().load_csv()
# cmp = customCompression(np.uint8)
cmp = customFreq()

# cmp = FP8()
# cmp = QSGD(2,True)
m_8 = MNIST_PAQ_COMP(cmp = cmp, filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", time.time()-tic)

  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_22056\1354010134.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30103}: 100%|██████████| 164/164 [00:36<00:00,  4.53it/s]


Test Accuracy: 0.10106
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 2, 'Loss': 2.05589}: 100%|██████████| 164/164 [00:28<00:00,  5.68it/s]


Test Accuracy: 0.27118
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 3, 'Loss': 1.78047}: 100%|██████████| 164/164 [00:29<00:00,  5.61it/s]


...

Test Accuracy: 0.90833
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 9, 'Loss': 1.54572}: 100%|██████████| 164/164 [00:36<00:00,  4.54it/s]


Test Accuracy: 0.91517
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 10, 'Loss': 1.53683}: 100%|██████████| 164/164 [00:36<00:00,  4.55it/s]


Test Accuracy: 0.92127
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  350.4498794078827


In [ ]:
{'Epoch': 1, 'Loss': 2.30103}: 100%|██████████| 164/164 [00:36<00:00,  4.53it/s]
Test Accuracy: 0.10106
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.05589}: 100%|██████████| 164/164 [00:28<00:00,  5.68it/s]
Test Accuracy: 0.27118
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.78047}: 100%|██████████| 164/164 [00:29<00:00,  5.61it/s]
Test Accuracy: 0.67313
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.64235}: 100%|██████████| 164/164 [00:28<00:00,  5.67it/s]
Test Accuracy: 0.83035
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.59154}: 100%|██████████| 164/164 [00:33<00:00,  4.87it/s]
Test Accuracy: 0.87038
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.56933}: 100%|██████████| 164/164 [00:37<00:00,  4.33it/s]
Test Accuracy: 0.8855
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.55506}: 100%|██████████| 164/164 [00:34<00:00,  4.79it/s]
Test Accuracy: 0.89718
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.54833}: 100%|██████████| 164/164 [00:35<00:00,  4.65it/s]
Test Accuracy: 0.90833
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.54572}: 100%|██████████| 164/164 [00:36<00:00,  4.54it/s]
Test Accuracy: 0.91517
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.53683}: 100%|██████████| 164/164 [00:36<00:00,  4.55it/s]
Test Accuracy: 0.92127
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  350.4498794078827

# Uint16-custom2


{'Epoch': 1, 'Loss': 2.30095}: 100%|██████████| 164/164 [00:59<00:00,  2.74it/s]
Test Accuracy: 0.09963
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.09561}: 100%|██████████| 164/164 [00:57<00:00,  2.84it/s]
Test Accuracy: 0.36056
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.86639}: 100%|██████████| 164/164 [00:58<00:00,  2.81it/s]
Test Accuracy: 0.63837
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.77721}: 100%|██████████| 164/164 [00:56<00:00,  2.90it/s]
Test Accuracy: 0.68152
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.74531}: 100%|██████████| 164/164 [01:01<00:00,  2.68it/s]
Test Accuracy: 0.71413
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.72985}: 100%|██████████| 164/164 [00:44<00:00,  3.71it/s]
Test Accuracy: 0.73011
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.64313}: 100%|██████████| 164/164 [01:02<00:00,  2.63it/s]
Test Accuracy: 0.73162
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.54832}: 100%|██████████| 164/164 [00:42<00:00,  3.83it/s]
Test Accuracy: 0.90883
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.53205}: 100%|██████████| 164/164 [00:22<00:00,  7.37it/s]
Test Accuracy: 0.92051
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.52779}: 100%|██████████| 164/164 [00:40<00:00,  4.01it/s]
Test Accuracy: 0.92549
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784

# uint8-custom 1
{'Epoch': 1, 'Loss': 2.30106}: 100%|██████████| 164/164 [00:54<00:00,  3.00it/s]
Test Accuracy: 0.10106
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.13939}: 100%|██████████| 164/164 [00:44<00:00,  3.65it/s]
Test Accuracy: 0.38529
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.98048}: 100%|██████████| 164/164 [00:23<00:00,  6.95it/s]
Test Accuracy: 0.44234
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.82871}: 100%|██████████| 164/164 [00:43<00:00,  3.76it/s]
Test Accuracy: 0.64131
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.77463}: 100%|██████████| 164/164 [00:25<00:00,  6.49it/s]
Test Accuracy: 0.68517
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.75027}: 100%|██████████| 164/164 [00:24<00:00,  6.64it/s]
Test Accuracy: 0.70226
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.73521}: 100%|██████████| 164/164 [00:47<00:00,  3.49it/s]
Test Accuracy: 0.71979
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.71573}: 100%|██████████| 164/164 [00:44<00:00,  3.67it/s]
Test Accuracy: 0.73043
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.68576}: 100%|██████████| 164/164 [00:53<00:00,  3.06it/s]
Test Accuracy: 0.73663
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.64119}: 100%|██████████| 164/164 [00:21<00:00,  7.65it/s]
Test Accuracy: 0.8215
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  399.71174716949463